In [1]:
## - My Strucutral BMP Modeling Notes
#### Notes on effort to develop a model that identifies structural BMPs for City facilities to reduce their storm water quality pollution risks.

In [2]:
#### sqlite basics:  https://pythonspot.com/python-database-programming-sqlite-tutorial/


import sqlite3 as lite
import sys
 
con = None
 
try:
    con = lite.connect('test.db')
    cur = con.cursor()    
    cur.execute('SELECT SQLITE_VERSION()')
    data = cur.fetchone()
    print ("SQLite version: %s" % data)                
except lite.Error as e:   
    print ("Error %s:" % e.args[0])
    sys.exit(1)
finally:    
    if con:
        con.close()
        

SQLite version: 3.14.2


In [12]:
#### how do you catch errors?
#### https://docs.python.org/3/tutorial/errors.html
#### list of standard python exceptions here: https://docs.python.org/3/library/exceptions.html#exception-hierarchy
#### you must know which error in list you excpect and look out for it explicitly. f.e. need to catch div 0 exception eplicitly:

x=0
try:
    1/x
except ZeroDivisionError as err:
    print (err)
    
    

division by zero


In [13]:
#### LET'S START BUILDING SQL TABLE MAKING SCRIPT

import sqlite3 as sqlite
import os

dbfilename = '_jonhonda_dat\\special_prj\\StrBMPModelDB003'
scriptfilename = '_jonhonda_dat\\special_prj\\MakeStrBMPModelTables'

print ("\nClearing old DB")
try:
    os.remove(dbfilename)
except FileNotFoundError as err:
    print ("no need to remove db file")####it's okay if file doesn't exist. ####
    
print ("Opening DB")
print (dbfilename)
connection = sqlite.connect(dbfilename)
connection.execute ("PRAGMA foreign_keys = ON")
cursor = connection.cursor()
# cursor.execute ('''DROP TABLE facilities''')
# connection.commit()
# cursor.execute ('''CREATE TABLE facilities (facility_id INTEGER PRIMARY KEY, facility_name TEXT, wrs_pollutant_risks_id INTEGER)''')
# connection.commit()

print ("Reading Script...")
scriptFile = open(scriptfilename, 'r')
script = scriptFile.read()
scriptFile.close()

print ("Running Script...")
cursor.executescript(script)

connection.commit()
print ("Changes successfully committed ...")
 
# cursor.execute('''INSERT INTO facilities (facility_name, wrs_pollutant_risks_id) VALUES(?,?)''', ('testfac',10))
# cursor.execute('''INSERT INTO base_bmps(bmp_name,bmp_removal_rates_id,cip_expression_id,om_expression_id,facility_id) VALUES(?,?,?,?,?)''',('testbmp',1,1,1,1))
# cursor.execute('''SELECT * FROM base_bmps''')
# print(cursor.fetchall())
# cursor.execute('''SELECT * FROM facilities''')
# print(cursor.fetchall())
# cursor.execute('''DELETE FROM facilities''')
# cursor.execute('''SELECT * FROM base_bmps''')
# print(cursor.fetchall())
connection.close()



Clearing old DB
Opening DB
_jonhonda_dat\special_prj\StrBMPModelDB003
Reading Script...
Running Script...
Changes successfully committed ...


In [14]:
#### SOME AUXHILIARY SQL FUNCTIONS:
def getDBConnection(dbFileName):
    # make a db connection
    import sqlite3 as sqlite
    #     dbfilename = '_jonhonda_dat\\special_prj\\StrBMPModelDB003'
    dbConnection = sqlite.connect(dbFileName)
    return dbConnection


def ExecuteSQL(dbConnection, sql_str, ParamTuple,flgCommitAfterTransaction=True):
    #execute insert/update/delete and other action type sql commands on the given db
    # attempt to return row that was updated
    #dbConnection either a path or sqlite.Connection object. if path, then db only open during function call
    #Optional Args:
        #fglCommitAfterTrans: false if you are going to commit later. true if you want to commit now.
                #NOTE: CAUTION!!!! set = True or leave out if you are using dbFileName option b/c db closes after           
    import sqlite3 as sqlite
#     print (sql_str)
#     print (ParamTuple)
    if type(dbConnection) == str:
        dbConnection = getDBConnection (dbConnection) #open db if given filename. otherwise, assume dbConnection passed in as dbObject
    cursor = dbConnection.cursor()    
    cursor.execute (sql_str, ParamTuple)
    lastRowID = cursor.lastrowid
    if flgCommitAfterTransaction: dbConnection.commit()
    if type(dbConnection) ==str: dbConnection.close() #then we were passed db only stays open during this routine
    return lastRowID

def QrySQL(dbConnection, sql_str, ParamTuple):
    #qry db and return results as list
    #Optional Args:
        #You must include either dbFileName (a path to the db you want to operate on)
                            #or dbObj (an open database object)
                        #if you use dbFileName, then db is open only during function call
        #fglCommitAfterTrans: false if you are going to commit later. true if you want to commit now.
                #NOTE: CAUTION!!!! set = True or leave out if you are using dbFileName option b/c db closes after       
    import sqlite3 as sqlite
    if type(dbConnection) == str:
        dbConnection = getDBConnection (dbConnection) #open db if given filename. otherwise, assume dbConnection passed in as dbObject
    cursor = dbConnection.cursor() 
    print (sql_str)
    print (ParamTuple)
    cursor.execute (sql_str, ParamTuple)
    rows = cursor.fetchall()
    if type(dbConnection) == str: dbConnection.close() #close db if db opened w/in this function
    return rows
    
def getRecordID(dbConnection, dbTableName, recordIDFieldName, WhereClause, WhereValTPL):
#retrieve record ID if it exists, or -1234 if DNE
#WhereClause: SQL string of Field equivalence and paramter eg: expression_name != (?) AND field_name = (?)
#WhereValTPL: tuple of values corresponding to the WhereClause

#     dbConnection = getDBConnection (dbFileName)
#     cursor = dbConnection.cursor()
#     sql_str = 'SELECT ' + recordIDFieldName + ' FROM ' + dbTableName+ ' WHERE ' + QryOnFieldName + ' = \'''' +  QryValue + '''\''''
#     print(sql_str)
#     cursor.execute(sql_str)
    row_id = -1234
    if len(WhereClause) >0:
        sql_str = 'SELECT ' + recordIDFieldName + ' FROM ' + dbTableName + ' WHERE ' + WhereClause
#         print (sql_str)
#         print (WhereValTPL)
        rows = QrySQL(dbConnection, sql_str, WhereValTPL)
        for row in rows:
            row_id = row[0]
    #         print (row_id)
            break 
    #     dbConnection.close()
    return row_id

def setRecord(dbConnection, dbTableName, RecordFieldLS, RecordValsTPL, PrimaryKeyField, WhereClause, WhereValTPL,flgCommitAfterTransaction=True):
    #insert or update into tableName:
        #RecordFieldLS: list of fields we want to insert/update record
        #RecordValsTPL: tuple of values corresponding to the RecordFieldLS list
        #PrimaryKeyField: table's primary key field name
        #WhereClause: fields we should query when determing if record exists;
                    #and if record exists, identifies what record should be updated
            #WhereClause: SQL string of Field equivalence and paramter eg: expression_name != (?), field_name = (?)
        #WhereVal: tuple of values corresponding to the where field
        # return tuple: (funcStatus, VarDict)
        #func status is list showing func status: [0] = 0 or 1 (0 if fault, 1 if ok)
        #                                         [1] = fault description  
    #dbConnection either a path or sqlite.Connection object. if path, then db only open during function call
    #Optional Args:
        #fglCommitAfterTrans: false if you are going to commit later. true if you want to commit now.
                #NOTE: CAUTION!!!! set = True or leave out if you are using dbFileName option b/c db closes after
            
#   determine if record already exists in db (-1234 if DNE)
#     WhereVal = str(WhereVal) #coerce to string no longer needed b/c we'ere using parameterized calls
    row_id=-1234
    row_id = getRecordID(dbConnection,dbTableName, PrimaryKeyField, WhereClause, WhereValTPL)
    if row_id < 0: #no record exists: 
        #build SQL field list:
        FieldStr =  ', ' .join(RecordFieldLS)
        FieldStr = '(' + FieldStr + ')'
        ValuesStr = ','.join(['?'] * len(RecordFieldLS)) #make values list
        ValuesStr = '(' + ValuesStr + ')'
        sql_str = 'INSERT INTO ' + dbTableName + FieldStr + ' Values' + ValuesStr
#         print ('Writing to table: ' + dbTableName + '   ' + sql_str)
        row_id = ExecuteSQL(dbConnection, sql_str, RecordValsTPL,flgCommitAfterTransaction)                    
    else: #exists
        #build SQL field list: def setRecord(dbFileName, dbTableName, RecordFieldLS, RecordValsTPL, PrimaryKeyField, WhereClause, WhereValTPL):
        FieldStr =  ' = (?), ' .join(RecordFieldLS) + ' = (?)'
        sql_str = 'UPDATE ' + dbTableName + ' SET ' + FieldStr + ' WHERE ' + WhereClause
        ParamTPL  = RecordValsTPL + WhereValTPL # create a new tuple that includes where clause's value
#         print ('Updating to table: ' + dbTableName + '   ' + sql_str)
        ExecuteSQL(dbConnection, sql_str, ParamTPL,flgCommitAfterTransaction) #execute SQL update   
    return row_id
   
def getFieldNames(dbFileName, TableName):
    #return field names from the given table
    sql_str = 'PRAGMA table_info('+TableName+')'
    ret = QrySQL(dbFileName, sql_str, ())
    return [row[1] for row in ret] #slice second column, which is the field name colum   
    
def getOpsValue (dbFileName, TableName, FieldName, OP, QryOnFieldName, QryOnLS):
    #build and execute SQL query to perform SQL math operation (SUM, MAX, MIN, etc) on FieldName 
    #QryOnFieldName: field name to query/constrain
    #QryOnList: list of values to qyery on
    questionmarks = '?' * len(QryOnLS)
    sql_str = 'SELECT ' + OP + '(' + FieldName + ') FROM ' + TableName + ' WHERE ' + QryOnFieldName + ' IN ({})'.format(','.join(questionmarks))  
    return QrySQL(dbFileName, sql_str, QryOnLS)

In [15]:
dbConnection = getDBConnection('_jonhonda_dat\\special_prj\\StrBMPModelDB003')
expression_id = getRecordID(dbConnection,'expressions','expression_id','expression_name=(?)',('Feas-1',))
print(expression_id)
dbConnection.close()

SELECT expression_id FROM expressions WHERE expression_name=(?)
('Feas-1',)
-1234


In [16]:
# # let's work on importing feasibility data into the db
# # how to read csv
# # https://docs.python.org/2/library/csv.html

# #expect values given as csv

# import csv
# with open('_jonhonda_dat\\special_prj\\Input_Files\\feasibility_test_questions.csv', 'r') as csvfile:
#     csvreader = csv.reader(csvfile,dialect='excel')
#     # expect and handle 1st row as header row
#     rowi = 0
#     for row in csvreader:
#         if rowi > 0:
#             for aitem in row:
#                 print(aitem)
#         rowi += 1

In [17]:
# functions to help with determining function status
# uses funcStatus list:
#func status is list showing func status: [0] = 0 or 1 (0 if fault, 1 if ok)
#                                         [1] = fault description
def isFuncStatusOK (funcStatus):
    #return 0 if fault, 1 if no fault
    return funcStatus[0]

def getFuncStatusFault (funcStatus):
    return funcStatus[1]

In [18]:
#functions to assist w/ expression and expression-variable declaration

def procInputVarDecs (VarDict, VarDecStr, WriteToTable, WriteToField, QryOnUniqueField, DataType):
#function to process variable declarations;
# return tuple: (funcStatus, VarDict)
#func status is list showing func status: [0] = 0 or 1 (0 if fault, 1 if ok)
#                                         [1] = fault description
# VarDict is the dictionary object that we will write to and return
# use the project standard VarDict: 
#    key = var name: [VarName, VarType, StoredTable, StoredField, QryOnUniqueField, DataType]
# VarDecStr is the string holding the csv variable declaration
#   expected format: var_type DELIMITER val
#           var_type: val; exp; dxp; val
#           DELIMITER: |
#           DECval:      var name; expr name; dynamic expr format:
#                dynamic_expr_format: dyn expr name(exprID_tablename~exprID_fieldname~unqFieldName)
#                identifies what table and field name holds reference to the expression_id, and the unique field  of the table that identifies the record. 


# WriteToTable & WriteToField are the table and field db_var type variable can be found at
# QryOnUniqueField is field that we should query on when trying to retrieve val from db table and db field
    funcStatus = [1,''] #start with okay status
    DELIMITER = '|'
    mySplitter = VarDecStr.split(DELIMITER) #split declaration into its 2 parts
    if len(mySplitter) != 2:
        if mySplitter[0] == 'N/A':
            return (funcStatus,VarDict)
        funcStatus = [0,'!!!!FAULT in procInputVarDecs: VarDecStr missing 2 parts: ' + str(VarDecStr)]
        return (funcStatus,VarDict)
    if WriteToField == 'USEDECVAL': #use declaration value as name for writing field
        WriteToField = mySplitter[1]
    if mySplitter[0] == 'var': #then it's value is held in the database
        VarDict =  add2VarDict(VarDict, mySplitter[1], 'val', WriteToTable,WriteToField, QryOnUniqueField, DataType)           
    elif mySplitter[0] == 'exp': #then it's value is defined by a static database held expression
        VarDict =  add2VarDict(VarDict, mySplitter[1], 'exp', 'expressions','expression_str', 'expression_id', DataType)
    elif mySplitter[0] == 'dxp': #then it's value is defined by a dynamically defined expression
        FullVarName = mySplitter[1] #'dyn expr name(exprID_tablename~exprID_fieldname~unqFieldName)'
        ParamPart = FullVarName[FullVarName.find('('):] #slice string to just the Parameter part, including open and close parens, 
#         dyn_expr_name = initstr[:len(initstr)-len(ParamPart)]
        ParamPart = ParamPart[1:len(ParamPart)-1] #revise ParamPart to exclude open and close parens
        ParamSplitter = ParamPart.split('~')
#         print ('my dxp: ' + dyn_expr_name)
        VarDict =  add2VarDict(VarDict, FullVarName, 'dxp', ParamSplitter[0],ParamSplitter[1], ParamSplitter[2], DataType)        
    elif mySplitter[0] != 'val': #the only other possible dec type is val. fault if not val
        funcStatus = [0,'!!!!FAULT in procInputVarDecs: Unknown var_type: ' + str(mySplitter[0])]
        return (funcStatus,VarDict)
    return (funcStatus,VarDict)

def add2VarDict (VarDict, VarName, VarType, StoredTable, StoredField, QryOnUniqueField, DataType):
# function to add variable definition to the given dictionary of variables
# use the project standard VarDict: 
#    key = var name: [VarName, VarType, StoredTable, StoredField, QryOnUniqueField, DataType]
    print('Adding to variable dictionary: ' + VarName)
    defList = [VarName, VarType, StoredTable, StoredField, QryOnUniqueField, DataType] #variable definition list
    key = VarName
    VarDict.update({key: defList})
    return VarDict

def registerExpr (dbFileName, expression_name, expression_str, VarDict):
#function to register expressions in db. 
#provide expression Str and var dict object
# assume any expressions in bar dict are already registered.
#Return expression id 
#determine if expression is in table
    import os
    import pickle
#     print ('Registering expression: ' + expression_name + '=' + expression_str)
# Pickle the 'data' dictionary using the highest protocol available.
# use the dumps command to write pickle to a string
    pickled_VarDict = pickle.dumps(VarDict, pickle.HIGHEST_PROTOCOL) ####uses the latest pickling version
#insert/update expression record, depending on if vars has data or not (if not, then it's assumed to be a const. val):    
    if len(VarDict) >0:
        RecordFieldLS = ['expression_name', 'expression_str', 'expression_data_type', 'vars']
        RecordValsTPL = (expression_name,expression_str,'FLOAT',pickled_VarDict)
    else:
        RecordFieldLS = ['expression_name', 'expression_str', 'expression_data_type']
        RecordValsTPL = (expression_name,expression_str,'FLOAT')        
    expression_id = setRecord(dbFileName, 'expressions', RecordFieldLS, RecordValsTPL, 'expression_id', 'expression_name = (?)', (expression_name,))
    return expression_id          

In [19]:
# initstr = 'dyn expr name x(exprID_tablename~exprID_fieldname~unqFieldName)'
# ParamPart = initstr[initstr.find('('):] #slice string to just the Parameter part, including open and close parens, 
# print (initstr[:len(initstr)-len(ParamPart)])
# ParamPart = ParamPart[1:len(ParamPart)-1] #revise ParamPart to exclude open and close parens
# print (ParamPart)
# ParamSplitter = ParamPart.split('~')
# print (ParamSplitter)


In [20]:
#EXPRESSION EVALUATOR:
import pickle
#### DEFINE GLOBALLY SCOPED CONSTANTS TO HELP UNDERSTAND WHAT ARRAY ELEMENT WE'RE ACCESSING:
####  Constants for expression record query tuple:
C_exprRecTuple_exprName = 0
C_exprRecTuple_exprStr = 1
C_exprRecTuple_exprDataType = 2
C_exprRecTuple_Vars = 3

C_VarDict_VarName = 0 #### Constant for Var Dict Key
#### Constants for Var Dict Array:
# use the project standard VarDict: 
#    key = var name: [VarName, VarType, StoredTable, StoredField, QryOnUniqueField, DataType]
C_VarDict_VarName = 0
C_VarDict_VarType = 1
C_VarDict_StoredTable = 2
C_VarDict_StoredField = 3
C_VarDict_QryOnUniqueField = 4
C_VarDict_DataType = 5


def EvalExpr(dbFileName,exprRecTuple, QryOnUnqFieldValsDict): #### pass in expression record as a tuple
    #record includes fields in this order:
        #SELECT expression_name, expression_str, expression_data_type, vars FROM expressions WHERE ... 
    #QryOnUnqFieldValsDict: dictionary of the value that value obtaining query should query against. FORMAT:
        #{key = table.fieldname: item = unique value}
        #this allows us to define, for example, the unique facility and base-bmp pair to query for:
            #{facility_chars.facility_id: 2,
            # base_bmps.bmp_name: 'hydrodynamic_separator'
            #}
    
    procstr = exprRecTuple[C_exprRecTuple_exprStr] #get expression string from record tuple
    if exprRecTuple[C_exprRecTuple_Vars] is None: #will be NoneType if no vars were entered to expression record (expression is probably a constant)
#         print ('empty')
        Vars= {} #make empty vars dictionary
    else:
        Vars = pickle.loads(exprRecTuple[C_exprRecTuple_Vars]) #unpickle to Vars variable (Dictionary type)
    print('proccessing expression: ' +  exprRecTuple[C_exprRecTuple_exprName] + '=' + procstr)
    for aVar in Vars.items(): #iterate thru each Var in Vars, replacing procstr's Var instances w/ Var's value 
        procstr = procstr.replace(aVar[C_VarDict_VarName],getVal(dbFileName, aVar,QryOnUnqFieldValsDict))
#         aVal = getVal(dbFileName, aVar,QryOnUnqFieldValsDict)
#         print ('Replace: ' + aVar[C_VarDict_VarName] + 'w ' + aVal + ' in ' + procstr)
#         procstr = procstr.replace (aVar[C_VarDict_VarName],aVal)
#         print ('Result ' + procstr)
        print('proccessing expression: ' +  exprRecTuple[C_exprRecTuple_exprName] + '=' + procstr)
    myVal = eval(procstr)
    print ('  eval(' + str(procstr) + ')=' + str(myVal))
    return myVal

        
def getVal(dbFileName, aVar, QryOnUnqFieldValsDict): #retrieve DB value, or call expression evaluation of passed variable (expects tuple of expression Query)   
    #QryOnUnqFieldValsDict: dictionary of the value that value obtaining query should query against. FORMAT:
        #{key = table.fieldname: item = unique value}
        #this allows us to define, for example, the unique facility and base-bmp pair to query for:
            #{facility_chars.facility_id: 2,
            # base_bmps.bmp_name: 'hydrodynamic_separator'
            #}

    print('    attempting to retrieve value for: ', aVar[C_VarDict_VarName] )
    strdbVal = 'fault_if_still_this'
    
    if aVar[1][C_VarDict_VarType] == 'val': #### value is housed somewhere in database. get value
        #unpack serialized data:
        dbTableName = aVar[1][C_VarDict_StoredTable]
        dbFieldName = aVar[1][C_VarDict_StoredField]
        dbQryOnUniqueField = aVar[1][C_VarDict_QryOnUniqueField]
        UnqFieldValsDict_Key = dbTableName + '.' + dbQryOnUniqueField
#         print ('unq key ' + UnqFieldValsDict_Key)
        #find matching table-field in QryOnUnqFieldValsDict:
        try:
            QryOnUniqueFieldVal = QryOnUnqFieldValsDict[UnqFieldValsDict_Key]#[dbTableName + '.' + dbFieldName]
#             print (str(QryOnUnqFieldValsDict))
        except KeyError:
            print ('     FAULT!!!! While evaluating expression: DB stored table name + field name: ' + dbTableName + '.' + dbFieldName + ' for the var was not included with QryOnUnqFieldValsDict')
            return strdbVal
        sql_str = '''SELECT ''' + dbFieldName + ''' FROM ''' + dbTableName + ''' WHERE ''' + dbQryOnUniqueField + '''=''' + str(QryOnUniqueFieldVal)
        print('      QUERY:' + sql_str)       
        myRows = QrySQL(dbFileName, sql_str, ())
        dbVal = myRows[0][0] #### return record as value
        if type(dbVal) == str:
            strdbVal = '\'' + dbVal + '\'' #encapsulate in quotes so python eval reads as str and not var
        else: #assume numeric
            strdbVal = str(dbVal) #### cast to string
        print('       QUERY RESULT: ' + aVar[C_VarDict_VarName] + '=' + strdbVal)
    
    elif aVar[1][C_VarDict_VarType]=='exp':
        print ('      This is an expression. Prepare to re-enter EvalExpr...')
        dbTableName = 'expressions'
        dbFieldName = 'expression_name'
        sql_str = '''SELECT expression_name, expression_str, expression_data_type, vars \
        FROM ''' + dbTableName + ''' WHERE ''' + dbFieldName + '''= \'''' +  aVar[C_VarDict_VarName] + '''\''''
        print('      QUERY:' + sql_str)
        myRows = QrySQL(dbFileName, sql_str, ())
        dbVal = myRows[0][0] #### return record as value
        print('       Reentering EvalExpr....')
        strdbVal = str(EvalExpr(dbVal)) #### cast to string
    
    elif aVar[1][C_VarDict_VarType]=='dxp':
    #                dynamic_expr_format: dyn expr name(exprID_tablename~exprID_fieldname~unqFieldName)
    #                identifies what table and field name holds reference to the expression_id, and the unique field  of the table that identifies the record. 
        print ('     This is a dynamic expression. Query for static expression using provided unique identifiers')
        #unpack dxp info:
        dbTableName = aVar[1][C_VarDict_StoredTable]
        dbFieldName = aVar[1][C_VarDict_StoredField]
        dbQryOnUniqueField = aVar[1][C_VarDict_QryOnUniqueField]
        UnqFieldValsDict_Key = dbTableName + '.' + dbQryOnUniqueField
#         print ('unq key ' + UnqFieldValsDict_Key)
        #get ready to query for expression id:
        #get unique value to query on:
        try:
            QryOnUniqueFieldVal = QryOnUnqFieldValsDict[UnqFieldValsDict_Key]#[dbTableName + '.' + dbFieldName]
            print (str(QryOnUnqFieldValsDict))
        except KeyError:
            print ('     FAULT!!!! While evaluating expression: DB stored table name + field name: ' + dbTableName + '.' + dbFieldName + ' for the var was not included with QryOnUnqFieldValsDict')
            return strdbVal
        sql_str = 'SELECT ' + dbFieldName + ' FROM ' + dbTableName + ' WHERE ' + dbQryOnUniqueField + '=(?)' 
        print (sql_str + QryOnUniqueFieldVal)
        ParamTuple = (QryOnUniqueFieldVal,)
        myRows = QrySQL(dbFileName, sql_str, ParamTuple) #query for dynamic expression record
        if len(myRows) == 0:
            print ('     FAULT!!!! While evaluating expression: DB stored table name + field name: ' + dbTableName + '.' + dbFieldName + ' dxp exprssion query has no record')
            return strdbVal
        expression_id = myRows[0][0] #extract expression_id
        if expression_id == -1234:
            print ('     FAULT!!!! While evaluating expression: DB stored table name + field name: ' + dbTableName + '.' + dbFieldName + ' dxp exprssion query has no record')
            return strdbVal        
        #now query for static expression record:
        sql_str = 'SELECT expression_name, expression_str, expression_data_type, vars \
                    FROM expressions WHERE expression_id = (?)'
        ParamTuple = (expression_id,)
        myExpr = QrySQL (dbFileName, sql_str, ParamTuple)
        if len(myExpr)>0:
            myExpr = myExpr[0] #extract tuple from list
            print ('       dynamic expression: ' + aVar[0] + ' = ' + ' static expression: ' + myExpr[0])
            print ('       Reentering EvalExpr...')
            strdbVal = str(EvalExpr(dbFileName,myExpr, QryOnUnqFieldValsDict))        
    else:
        print (strdbVal)
    return strdbVal


In [21]:
# #playing w/ classes:
# # https://jeffknupp.com/blog/2014/06/18/improve-your-python-python-classes-and-object-oriented-programming/
# class Car(object):
    
    
#     wheels = 4
      
        
#     def __init__(self, make, model):
#         self.make = make
#         self.model = model
# #         static.add2CarCnt()
        
# mustang = Car('Ford', 'Mustang')
# print (mustang.wheels)
# # mustang.add2CarCnt()



In [22]:
#functions to assist w/ pollutant removal rate table inserts and updates
# defining as a class makes sense since there are a lot of pollutant types, and we will interact with them in many ways, and we will use them as basis for pollutant wrs component
from abc import ABCMeta, abstractmethod

class abc_Pollutants(object):  
# first, define pollutants as an abstract class (removal rates, wrs pollutants will both inherit from it)
    __metaclass__ = ABCMeta
  
    def __init__(self):
        self.dict_Pollutants = {'tss': 0,'turbidity': 0,'p': 0,
                           'n': 0, 'nn': 0, 'an':0,
                           'og': 0,
                           'cu': 0, 'zn':0, 'fe': 0,
                           'phmin': 0, 'phmax': 0                          
                          }
    
    def GetPollutantVal(self, PollutantName):
        return self.dict_Pollutants[PollutantName]
    
    def SetPollutantVal(self, PollutantName, mVal):
        self.dict_Pollutants[PollutantName] = mVal

    def GetAllPollutantNames(self):
        return list(self.dict_Pollutants.keys())
    
    def GetAllPollutantVals(self):
        return list(self.dict_Pollutants.values())
        
class cls_Pollutant_Removal_Rates(abc_Pollutants):
    def clsType(self):
        return 'pollutant_removal_rates'
    
#     def getPRR_From_DB(self,PRR_id):
#     #function to retrieve pollutant removal record of the given pollutant_Removal_Rates_id
#     #set object's values equal to retrieved removal rates
    
    
    def registerPRRDB(self, dbFileName, PRR_id):
    #function to register pollutant removal rates in pollutant removal rate table of DB. either inserts or updates
    #if id given for record in DB table: pollutant_removal_rates_id then update,
    #if no id given (i.e. passed -1234) then insert
    # return tuple: (funcStatus, PRR_id)
    #func status is list showing func status: [0] = 0 or 1 (0 if fault, 1 if ok)
    #                                         [1] = fault description
        funcStatus = [1,''] #start with okay status
    # append r_ to all Pollutant Names to get table field names:
        PollutantNames = self.GetAllPollutantNames() #get list of pollutants
        for index in range(len(PollutantNames)):
            PollutantNames[index] = 'r_' + PollutantNames[index]
    #now write/update record:
        RecordFieldLS = PollutantNames
        RecordValsTPL = tuple(self.GetAllPollutantVals())
        PRR_id = setRecord(dbFileName, 'pollutant_removal_rates', RecordFieldLS, RecordValsTPL, 'pollutant_removal_rates_id','pollutant_removal_rates_id=(?)', (PRR_id,))       
        return  (funcStatus,PRR_id)
        
# prr = cls_Pollutant_Removal_Rates()
# prr.SetPollutantVal('tss', 5000)
# prr.SetPollutantVal('an',2.05)
# # print (prr.GetPollutantVal('tss'))
# # print (prr.GetAllPollutantNames()) 
# # print (prr.GetAllPollutantVals())
                   
                   
# ID = prr.registerPRRDB('_jonhonda_dat\\special_prj\\StrBMPModelDB003',8)

# print (ID[1])

In [23]:
def importFeasibilityQuestionsCSV(dbFileName):
    import csv  
    #define csv column constants:
    C_fid_element = 0
    C_english_element = 1
    C_FacAvailableValVar = 2
    C_BMPReqdValVar = 4
    C_expression_element = 5
    print ('Reading csv for import to Feasibility Questions')
    with open('_jonhonda_dat\\special_prj\\Input_Files\\feasibility_test_questions.csv', 'rt', encoding='utf-8-sig') as csvfile:
        csvreader = csv.reader(csvfile,dialect='excel')
        # expect and handle 1st row as header row
        rowi = 0
        for row in csvreader:
            if rowi > 0: 
                #read expression definition & write expression to expressions table
                Fid = row[C_fid_element]
                print ('\nReading csv record: ' + Fid)
                Question_english = row[C_english_element]
                Question_expression = row[C_expression_element]
                VarDict = {} #init empty VarDict for expression
    #             build variable def. for db_val types, assume val obtained from fac. char table in db_val field, w/ fac_id as unique queryable field            
    #             eval type based on type given. expect passed value to be val_type.value
                retStatus = procInputVarDecs(VarDict, row[C_FacAvailableValVar], 'facility_chars', 'USEDECVAL', 'facility_id', 'FLOAT')
                if isFuncStatusOK(retStatus[0]) == 1: # check if return status ok
                    retStatus= procInputVarDecs(VarDict, row[C_BMPReqdValVar], 'facility_chars', 'USEDECVAL', 'facility_id', 'FLOAT')
                    VarDict = retStatus[1]
                if isFuncStatusOK(retStatus[0]) == 0: #check if return status ok
                    print (getFuncStatusFault(retStatus[0]) + '    Fix error and retry.')
                else:
                    VarDict = retStatus[1]
        #          now write expression information:
                    expression_id = registerExpr (dbFileName,Fid, Question_expression, VarDict)
                    #now write feasibility_test_question record:
                    RecordFieldLS = ['feas_id', 'question_english', 'question_expression_id']
                    RecordValsTPL = (Fid,Question_english,expression_id)
                    feasibility_test_question_id = setRecord(dbFileName, 'feasibility_test_questions', RecordFieldLS, RecordValsTPL, 'feasibility_test_question_id','feas_id=(?)',(Fid,))
            rowi +=1
importFeasibilityQuestionsCSV('_jonhonda_dat\\special_prj\\StrBMPModelDB003')

Reading csv for import to Feasibility Questions

Reading csv record: Feas-1
Adding to variable dictionary: OFFSITE_SD_Exist
SELECT expression_id FROM expressions WHERE expression_name = (?)
('Feas-1',)
SELECT feasibility_test_question_id FROM feasibility_test_questions WHERE feas_id=(?)
('Feas-1',)

Reading csv record: Feas-2
Adding to variable dictionary: GW_Risk
SELECT expression_id FROM expressions WHERE expression_name = (?)
('Feas-2',)
SELECT feasibility_test_question_id FROM feasibility_test_questions WHERE feas_id=(?)
('Feas-2',)

Reading csv record: Feas-3
Adding to variable dictionary: GW_Risk
SELECT expression_id FROM expressions WHERE expression_name = (?)
('Feas-3',)
SELECT feasibility_test_question_id FROM feasibility_test_questions WHERE feas_id=(?)
('Feas-3',)

Reading csv record: Feas-4
Adding to variable dictionary: Soil_Type
SELECT expression_id FROM expressions WHERE expression_name = (?)
('Feas-4',)
SELECT feasibility_test_question_id FROM feasibility_test_questions

In [24]:
#### build base bmp importer:
def importBaseBMPsCSV(dbFileName):
    import csv  
    #define csv column constants:
    C_bmp_id = 0  #bmp id used in import file. not necessarily same id as base_bmp_id
    C_bmp_name = 1
    C_cip_cost_unit = 2
    C_om_cost_unit = 3
    C_bmp_size_unit = 4
    C_cip_cost_expr = 5
    C_om_cost_expr = 6
    C_bmp_size_expr = 7
    C_tss = 8
    C_turb = 9
    C_og = 10
    C_fe = 11
    C_cu = 12
    C_zn = 13
    C_n = 14
    C_p = 15
    C_nn = 16
    C_an = 17    
    C_phmax = 18
    C_feastest_StartCol = 19 #1st column holding feasibility test. assume remaining rows are all feasibility tests, identified by column name 
    print ('Reading csv for import to base bmp tables')
    with open('_jonhonda_dat\\special_prj\\Input_Files\\bmp_lego_piece.csv', 'rt', encoding='utf-8-sig') as csvfile:
        csvreader = csv.reader(csvfile,dialect='excel')
        # expect and handle 1st row as header row
        rowi = 0
        for row in csvreader:
            if rowi == 0:
                csvFieldLS = row
            if rowi > 0: 
                bmp_name = row[C_bmp_name] #get BMP name from CSV row
                print ('\nReading csv record: ' + bmp_name)                
                #write cip cost expression. if var, then assume value can be found in facility_chars table
                print ('Reading csv cip cost info...')
                cip_cost_expr = row[C_cip_cost_expr]
                VarDict = {} #init empty VarDict for expression
                retStatus = procInputVarDecs(VarDict, row[C_cip_cost_unit], 'facility_chars', 'USEDECVAL', 'facility_id', 'FLOAT')
                if isFuncStatusOK(retStatus[0]) == 0: #check if return status ok
                    print (getFuncStatusFault(retStatus[0]) + '    Fix error and retry.')
                    cip_cost_expr_id=-1234
                else:              
                    cip_cost_expr_id = registerExpr (dbFileName,'cip_cost_expr_' + bmp_name, cip_cost_expr, VarDict)#now write expression information:

                #write om cost expression. if var, then assume value can be found in facility_chars table
                print ('Reading csv o&m cost info...')
                om_cost_expr = row[C_om_cost_expr]
                VarDict = {} #init empty VarDict for expression
                retStatus = procInputVarDecs(VarDict, row[C_om_cost_unit], 'facility_chars', 'USEDECVAL', 'facility_id', 'FLOAT')
                if isFuncStatusOK(retStatus[0]) == 0: #check if return status ok
                    print (getFuncStatusFault(retStatus[0]) + '    Fix error and retry.')
                    om_cost_expr_id = -1234
                else:              
                    om_cost_expr_id = registerExpr (dbFileName,'om_cost_expr_' + bmp_name, om_cost_expr, VarDict)#now write expression information:
                
                #write bmp sizing expression. if var, then assume value can be found in facility_chars table
                print ('Reading csv bmp sizing info...')
                bmp_size_expr = row[C_bmp_size_expr]
                VarDict = {} #init empty VarDict for expression
                retStatus = procInputVarDecs(VarDict, row[C_bmp_size_unit], 'facility_chars', 'USEDECVAL', 'facility_id', 'FLOAT')
                if isFuncStatusOK(retStatus[0]) == 0: #check if return status ok
                    print (getFuncStatusFault(retStatus[0]) + '    Fix error and retry.')
                    bmp_size_expr_id = -1234
                else:        
#                     print ('got here' + bmp_size_expr)
                    bmp_size_expr_id = registerExpr (dbFileName,'bmp_size_expr_' + bmp_name, bmp_size_expr, VarDict) #now write expression information:
            
                #write pollutant removal rates
                print ('Reading pollutant removal rate info...')
                prr = cls_Pollutant_Removal_Rates() #instantiate pollutant_Removal_Rate object
                prr.SetPollutantVal('tss', row[C_tss])
                prr.SetPollutantVal('turbidity', row[C_turb])
                prr.SetPollutantVal('og', row[C_og])
                prr.SetPollutantVal('fe', row[C_fe])
                prr.SetPollutantVal('cu', row[C_cu])
                prr.SetPollutantVal('zn', row[C_zn])
                prr.SetPollutantVal('n', row[C_n])
                prr.SetPollutantVal('p', row[C_p])
                prr.SetPollutantVal('nn', row[C_nn])
                prr.SetPollutantVal('an', row[C_an])
                prr.SetPollutantVal('phmax', row[C_phmax])
#                 print (prr.dict_Pollutants)
                retStatus = prr.registerPRRDB(dbFileName, -1234) #attempt pollutant removal rate registration  
                if isFuncStatusOK(retStatus[0]) == 1: # check if return status ok
                    PRR_id = retStatus[1] 
                else:
                    print ('Error while registering pollutant removal rates: ' + getFuncStatusFault(retStatus[0]) + '    Fix error and retry.')
                
                #we now have information needed to create base_bmp record
                RecordFieldLS = ['bmp_name', 'bmp_removal_rates_id', 'cip_expression_id', 'om_expression_id', 'bmp_size_expression_id']
                RecordValsTPL = (bmp_name, PRR_id, cip_cost_expr_id, om_cost_expr_id, bmp_size_expr_id)
                base_bmp_id=-1234
                base_bmp_id = setRecord(dbFileName, 'base_bmps', RecordFieldLS, RecordValsTPL, 'base_bmp_id','bmp_name = (?)',(bmp_name,))
            
            #use base bmp record to create bmp_feasibility_test_definition:
                #check that we have base_bmp record:
                if base_bmp_id > 0:
                    print ('Linking feasibility tests w/ base bmp: ' + str(base_bmp_id))
                    print ('Clear feasibility test definitions for the base bmp')
                    sql_str = 'DELETE FROM base_bmp_feasibility_test_definitions WHERE base_bmp_id = (?)'
                    ParamTPL = (base_bmp_id,)
                    ExecuteSQL(dbFileName, sql_str, ParamTPL) #execute SQL update 
                    #if feas test at csv column is not N/A then add record 
                    for csvCol in range(C_feastest_StartCol, len(row)):
                        if row[csvCol] != 'N/A':                          
                            ftq_id = getRecordID(dbFileName, 'feasibility_test_questions', 'feasibility_test_question_id' , 'feas_id=(?)', (csvFieldLS[csvCol],))
                            RecordFieldLS = ['feasibility_test_question_id', 'base_bmp_id']                           
                            RecordValsTPL = (ftq_id, base_bmp_id)
                            setRecord(dbFileName, 'base_bmp_feasibility_test_definitions', RecordFieldLS, RecordValsTPL, 'base_bmp_feasibility_test_definition_id','base_bmp_feasibility_test_definition_id=(?)', (-1234,))
            rowi += 1

importBaseBMPsCSV('_jonhonda_dat\\special_prj\\StrBMPModelDB003')    

Reading csv for import to base bmp tables

Reading csv record: Hydrodynamic Separation
Reading csv cip cost info...
Adding to variable dictionary: WQFR
SELECT expression_id FROM expressions WHERE expression_name = (?)
('cip_cost_expr_Hydrodynamic Separation',)
Reading csv o&m cost info...
Adding to variable dictionary: WQFR
SELECT expression_id FROM expressions WHERE expression_name = (?)
('om_cost_expr_Hydrodynamic Separation',)
Reading csv bmp sizing info...
SELECT expression_id FROM expressions WHERE expression_name = (?)
('bmp_size_expr_Hydrodynamic Separation',)
Reading pollutant removal rate info...
SELECT pollutant_removal_rates_id FROM pollutant_removal_rates WHERE pollutant_removal_rates_id=(?)
(-1234,)
SELECT base_bmp_id FROM base_bmps WHERE bmp_name = (?)
('Hydrodynamic Separation',)
Linking feasibility tests w/ base bmp: 1
Clear feasibility test definitions for the base bmp
SELECT feasibility_test_question_id FROM feasibility_test_questions WHERE feas_id=(?)
('Feas-1',)
SEL

SELECT base_bmp_id FROM base_bmps WHERE bmp_name = (?)
('Bioinfiltration / Bioretention',)
Linking feasibility tests w/ base bmp: 5
Clear feasibility test definitions for the base bmp
SELECT feasibility_test_question_id FROM feasibility_test_questions WHERE feas_id=(?)
('Feas-1',)
SELECT base_bmp_feasibility_test_definition_id FROM base_bmp_feasibility_test_definitions WHERE base_bmp_feasibility_test_definition_id=(?)
(-1234,)
SELECT feasibility_test_question_id FROM feasibility_test_questions WHERE feas_id=(?)
('Feas-2',)
SELECT base_bmp_feasibility_test_definition_id FROM base_bmp_feasibility_test_definitions WHERE base_bmp_feasibility_test_definition_id=(?)
(-1234,)
SELECT feasibility_test_question_id FROM feasibility_test_questions WHERE feas_id=(?)
('Feas-5',)
SELECT base_bmp_feasibility_test_definition_id FROM base_bmp_feasibility_test_definitions WHERE base_bmp_feasibility_test_definition_id=(?)
(-1234,)
SELECT feasibility_test_question_id FROM feasibility_test_questions WHERE f

SELECT base_bmp_id FROM base_bmps WHERE bmp_name = (?)
('Paving and Curbing',)
Linking feasibility tests w/ base bmp: 10
Clear feasibility test definitions for the base bmp
SELECT feasibility_test_question_id FROM feasibility_test_questions WHERE feas_id=(?)
('Feas-17',)
SELECT base_bmp_feasibility_test_definition_id FROM base_bmp_feasibility_test_definitions WHERE base_bmp_feasibility_test_definition_id=(?)
(-1234,)

Reading csv record: Oil and Water Separators
Reading csv cip cost info...
Adding to variable dictionary: WQFR
SELECT expression_id FROM expressions WHERE expression_name = (?)
('cip_cost_expr_Oil and Water Separators',)
Reading csv o&m cost info...
Adding to variable dictionary: WQFR
SELECT expression_id FROM expressions WHERE expression_name = (?)
('om_cost_expr_Oil and Water Separators',)
Reading csv bmp sizing info...
Adding to variable dictionary: WQFR
SELECT expression_id FROM expressions WHERE expression_name = (?)
('bmp_size_expr_Oil and Water Separators',)
Reading

In [25]:
#### strange non-space char at beginning of CSV file.
#### upon investigating I found the ascii value is 65279. i found out this is somethign called a byte order mark.
#### I was using the following code to read in csv:
# import csv
# with open('_jonhonda_dat\\special_prj\\Input_Files\\facility_chars.csv', 'rt', encoding='utf-8') as csvfile:

#### according to https://stackoverflow.com/questions/13590749/reading-unicode-file-data-with-bom-chars-in-python
#### to remove byte order mark, we should use the encoding 'utf-8-sig'
### seems to have worked.

In [26]:
####  build the facility characteristics importer
#### DEPRECATED BY GENEREIC CSV LOADER
# def importFacilityChars():
#     import sqlite3 as sqlite
#     import csv
#     #assume all csv fields match database field names
#     #assume all columns in csv file are intended to be inserted into fac char table
#     dbfilename = '_jonhonda_dat\\special_prj\\StrBMPModelDB003'
#     connection = sqlite.connect(dbfilename)
#     cursor = connection.cursor()    
#     with open('_jonhonda_dat\\special_prj\\Input_Files\\facility_chars.csv', 'rt', encoding='utf-8-sig') as csvfile:
#         csvreader = csv.reader(csvfile,dialect='excel')
#         # expect and handle 1st row as header row. add headers to list
#         print ('Reading csv file for facility_chars table input')
#         field_namesLS=[]
#         dataVals = ()
# #         print (type(csvreader))
#         rowi = 0
#         valuesStr = ''
#         sql_str = ''
#         for row in csvreader:
# #             print (type(row))
#             if rowi==0:
#                 field_namesLS = row #write field names as a list            
#                 #input data if field names found
#                 if len(field_namesLS) > 0: 
#                     field_namesStr = ', '.join(field_namesLS) #make field list                  
#                     field_namesStr = '(' + field_namesStr + ')'
#                     valuesStr = ','.join(['?'] * len(field_namesLS)) #make values list
#                     valuesStr = '(' + valuesStr + ')'
#                     sql_str = 'INSERT INTO facility_chars' + field_namesStr + ' VALUES' + valuesStr
#                     print ('Read in facility_char headers from csv')                    
#             else:
#                 rowTpl = tuple(row)
# #                 print (type(row))
#                 try:
#                     cursor.execute (sql_str, rowTpl)
#                     connection.commit() 
#                     print ('Wrote facility_char input to table')
#                 except sqlite.IntegrityError as err:
#                     print ('error encountered while inserting row ' + str(rowi) + ' into table. err: ' + str(err))
#             rowi +=1     
#     connection.close()
    
# importFacilityChars()

In [27]:
####  generalize facility csv importer to handle 'any' csv:
### good for situations where you can just import csv 'as is' w/o need for processing individual columns.
def importCSV2DB(csvFileName, dbFileName, TableName, PrimaryKeyField, WhereField, UseCSVWhereVal, RetAssocFieldID=False, flgCommitAfterTransaction=True):
    import csv
    #import data in csv at fPath
    # write to db at dbPath
    # insert into TableName
    #PrimaryKeyField: Name of table's primary key field
    #Where field: field name to check record existance on
    #UseCSVWhereVal: if True, then check if record exists in TableName by querying against the WhereField valued csv column. 
                     #if != True, then use given value in where clause 
    #assume all csv fields match database field names
    #if RetAssocFieldID !=False then user requesting to associate the csv record with a database record held in TableName, FieldName
        #Assume passed parameter LIST:
            #[TableName,FieldName,AssocFieldName,AssocFieldName,CSVAssocColName]
    C_TableName = 0 #TableName & FieldName = name of table and field holding database record id associated w/ csv record
    C_FieldName = 1
    C_AssocFieldName = 2 #AssocFieldName = field name to do where clause on (e.g. WHERE AssocFieldName = (?))
    C_CSVAssocColName = 3 #CSVAssocColName = name of CSV column whos values should be used to associate w/ the database table and field name
        #!!!IMPORTANT! the CSVAssocCol will not be included in csv import. only used for association.
    #return list of lists: if RetAssocFieldID == False: List[insert/update record_id]
                                #!= False: LIST of record pair[[insert/update record_id,assoc_record_id],[insert/update record_id,assoc_record_id]]
        
    with open(csvFileName, 'rt', encoding='utf-8-sig') as csvfile:
        csvreader = csv.reader(csvfile,dialect='excel')
        # expect and handle 1st row as header row. add headers to list
            
        print ('\nReading csv file for input to table: ' + TableName)
        field_namesLS=[]
        dataVals = ()
#         print (type(csvreader))
        rowi = 0
        valuesStr = ''
        sql_str = ''
        WhereFieldCol = 0
        RecordFieldLS = []
        ReturnLS = [] #return list
        for row in csvreader:
            if rowi==0:
                RecordFieldLS = row #write field names as a list 
                #'get field info'
                if UseCSVWhereVal == True:
                    try:
                        WhereFieldCol = RecordFieldLS.index(WhereField) #find WhereField column number
                    except IndexError:
                        print ('FAULT!!!! No csv fields found. Import failed')
                        return ReturnLS                  
                    except ValueError:
                        print ('!!!! FAULT! Unable to find WhereField: ' + str(WhereField) + ' column in CSV headers. Fix Error.')
                        return ReturnLS
                if RetAssocFieldID != False: #then we need to return record_id associated w/ some csv record. use value in CSVAssocFieldCol to determine association
                    try:
                        CSVAssocFieldCol = RecordFieldLS.index(RetAssocFieldID[C_CSVAssocColName]) #find csv column number that holds data we want to associate against
                    except ValueError:
                        print ('!!!! FAULT! Unable to find CSV column called: ' + str(RetAssocFieldID) + ' in CSV headers. Fix Error.')
                        return ReturnLS
                    del RecordFieldLS[CSVAssocFieldCol] #remove csv column from list  
            else:
                print ('Reading record at row: ' + str(rowi) + ' ' + str(row[WhereFieldCol]))
                #default csv association parameters:
                CSV_val = ''
                assoc_id = 0
                if RetAssocFieldID != False: # then function entered w/ request to return association w/ csv record
                    CSV_val = row[CSVAssocFieldCol]
                    assoc_id = getRecordID(dbConnection,RetAssocFieldID[C_TableName], RetAssocFieldID[C_FieldName], RetAssocFieldID[C_AssocFieldName] + '=(?)', (CSV_val,))
                    del row[CSVAssocFieldCol] #remove facility name colum from record
                RecordValsTPL = tuple(row)
                if assoc_id <0: #need to assoc w/ facility, but unsuccessfull attempt to obtain facility id information. skip record insertion
                    print ('Skipping CSV record.  The ' + RetAssocFieldID[C_CSVAssocColName]  + ' record: ' + CSV_val + ' does not exist in table ' + RetAssocFieldID[C_TableName])
                else:
                    if UseCSVWhereVal == True:
                        rec_id = setRecord(dbFileName, TableName, RecordFieldLS, RecordValsTPL, PrimaryKeyField, 
                                           WhereField + '=(?)', (row[WhereFieldCol],), flgCommitAfterTransaction)               
                    else:
                        rec_id = setRecord(dbFileName, TableName, RecordFieldLS, RecordValsTPL, PrimaryKeyField, 
                                           WhereField + '=(?)', (UseCSVWhereVal,), flgCommitAfterTransaction)                                       

                    if RetAssocFieldID == False:
                        ReturnLS += [[rec_id]] #no request for csv association
                    else:
                        ReturnLS += [[rec_id, assoc_id]] #request for csv assoc.
            rowi +=1
    return ReturnLS

def assocRecords (dbFileName, importLS, assocTable, assocField0, assocField1, flgCommitAfterTransaction):
    #associate record_id 0 in importLS list w/ record_id in a given table 
    #imporLS: list generated by importCSV2DB: List[[record0,record1],[record0,record1]] 
    #assocTable: the table that holds the record we want to do the association on 
    #assocField0: field that we want to set.
    #assocField1: field that we want to association on
        #i.e: UPDATE assocTable SET assocField0 = importLS[0] WHERE assocField1 = importLS[1]
    for importrec in importLS:
        setRecord(dbConnection, assocTable, [assocField0], (importrec[0],), assocField1, assocField1 + '=(?)', (importrec[1],), flgCommitAfterTransaction)                                   
        print ('Associated ' + assocTable + '.' + assocField0 + ': ' + str(importrec[0]) + ' with ' + assocField1 + ': ' + str(importrec[1]))
        
dbConnection = getDBConnection('_jonhonda_dat\\special_prj\\StrBMPModelDB003')
flgCommitAfterTransaction=False
#IMPORT BASIC FACILITY CHARS:
print ('\nImporting facility characteristics:')
importCSV2DB('_jonhonda_dat\\special_prj\\Input_Files\\facility_chars.csv', \
             dbConnection, 'facility_chars', 'facility_id', 'Fac_Name',True,False, False)

#IMPORT FACILITY RAINFALL EXTRACTED FROM http://rainfall.geography.hawaii.edu/downloads.html
print ('\nImporting Facility Rainfall Data:')
importLS = importCSV2DB('_jonhonda_dat\\special_prj\\Input_Files\\FacilityRainfallData.csv', \
             dbConnection, \
            'facility_monthly_rain', 'facility_monthly_rain_id', 
            'facility_monthly_rain_id',-1234, 
            ['facility_chars', 'facility_id', 'Fac_Name', 'Fac_Name'], 
            False)
assocRecords(dbConnection, importLS, 'facility_chars', 'facility_monthly_rain_id', 'Fac_Name' ,flgCommitAfterTransaction) #assoc w/ facility_id via facility_chars.facility_monthly_rain_id

#IMPORT FACILITY RISKS:
print ('\nImporting Facility Risks:')
importLS = importCSV2DB('_jonhonda_dat\\special_prj\\Input_Files\\facility_risks.csv', \
             dbConnection, 'facility_risks', 'facility_risk_id', 'facility_risk_id',-1234, 'Fac_Name', False)
assocRecords (dbConnection, importLS, 'facility_chars', 'existing_facility_risk_id',flgCommitAfterTransaction) #assoc w/ facility_id via facility_chars.existing_facility_risk_id

#IMPORT 
    
    
#IMPORT TBL1 NELS:
# print ('\nImporting Table 1 NELs:')
# importLS = importCSV2DB('_jonhonda_dat\\special_prj\\Input_Files\\facility_tbl1_nels.csv', \
#              dbConnection, \
#              'wrs_pollutant_risks', 'wrs_pollutant_risks_id', 'wrs_pollutant_risks_id',-1234, 'Fac_Name', False)
# #update facility_chars w/ facility risk id:
# for importrec in importLS:
#     rec_id = importrec[0] #wrs_id
#     fac_id = importrec[1] #associated facility id
#     setRecord(dbConnection, 'facility_risks', ['wrs_pollutant_limits_id'], (rec_id,), 'facility_id', 'facility_id' + '=(?)', (fac_id,), flgCommitAfterTransaction)                                   
#     print ('Added risk record for facility_id: ' + str(fac_id) + ' facility_risk_id: ' + str(rec_id))

    
# importFacRisks('_jonhonda_dat\\special_prj\\Input_Files\\facility_risks.csv', dbConnection, flgCommitAfterTransaction)
if flgCommitAfterTransaction==False: dbConnection.commit() #commit transactions at end if false
dbConnection.close()





Importing facility characteristics:

Reading csv file for input to table: facility_chars
Reading record at row: 1 Kalihi-Palama Bus & Paratransit Facility
SELECT facility_id FROM facility_chars WHERE Fac_Name=(?)
('Kalihi-Palama Bus & Paratransit Facility',)
Reading record at row: 2 Pearl City Bus Facility
SELECT facility_id FROM facility_chars WHERE Fac_Name=(?)
('Pearl City Bus Facility',)

Importing Facility Rainfall Data:

Reading csv file for input to table: facility_monthly_rain
Reading record at row: 1 Kailua Wastewater Treatment Plant
SELECT facility_id FROM facility_chars WHERE Fac_Name=(?)
('Kailua Wastewater Treatment Plant',)
Skipping CSV record.  The Fac_Name record: Kailua Wastewater Treatment Plant does not exist in table facility_chars
Reading record at row: 2 Wahiawa Wastewater Treatment Plant
SELECT facility_id FROM facility_chars WHERE Fac_Name=(?)
('Wahiawa Wastewater Treatment Plant',)
Skipping CSV record.  The Fac_Name record: Wahiawa Wastewater Treatment Plant d

SELECT facility_id FROM facility_chars WHERE Fac_Name=(?)
('Mililani Pump Station',)
Skipping CSV record.  The Fac_Name record: Mililani Pump Station does not exist in table facility_chars
Reading record at row: 166 Miomio Pump Station
SELECT facility_id FROM facility_chars WHERE Fac_Name=(?)
('Miomio Pump Station',)
Skipping CSV record.  The Fac_Name record: Miomio Pump Station does not exist in table facility_chars
Reading record at row: 167 Pacific Palisades Pump Station
SELECT facility_id FROM facility_chars WHERE Fac_Name=(?)
('Pacific Palisades Pump Station',)
Skipping CSV record.  The Fac_Name record: Pacific Palisades Pump Station does not exist in table facility_chars
Reading record at row: 168 Pearl City Pump Station
SELECT facility_id FROM facility_chars WHERE Fac_Name=(?)
('Pearl City Pump Station',)
Skipping CSV record.  The Fac_Name record: Pearl City Pump Station does not exist in table facility_chars
Reading record at row: 169 Punawai Pump Station
SELECT facility_id FRO

TypeError: assocRecords() missing 1 required positional argument: 'flgCommitAfterTransaction'

In [ ]:
ls = []
ls  += [[1]]
ls  += [2]
print (ls)

In [ ]:
#EVALUATE base_bmp feasibility:
def is_base_bmp_feasible(dbFileName, facility_id, base_bmp_id):
    #use db feasibility data to determine if base bmp is feasible for the given facility.
    print ('Determine feasibility of base bmp: ' + str(base_bmp_id) + '  at Facility: ' + str(facility_id))
    sql_str = 'SELECT bbftd.feasibility_test_question_id, ftq.feas_id, is_feasible, base_bmp_feasibility_test_result_id  \
                FROM base_bmp_feasibility_test_results AS bbftr \
                INNER JOIN base_bmp_feasibility_test_definitions AS bbftd ON bbftr.base_bmp_feasibility_test_definition_id = bbftd.base_bmp_feasibility_test_definition_id \
                INNER JOIN feasibility_test_questions AS ftq ON bbftd.feasibility_test_question_id = ftq.feasibility_test_question_id \
                WHERE facility_id = (?) AND base_bmp_id =(?)'
    ParamTuple = (facility_id,base_bmp_id)
    Tests = QrySQL (dbFileName, sql_str, ParamTuple)
    is_bb_feasible = 1 #start w/ bmp feasibility as true (necessary start condition for bool and ops to work correctly)
#     print ('using DB data:')
    for Test in Tests:
        print ('Feas_id: ' + str(Test[1]) + ' Test Result: ' +  str(Test[2]) + ' (bbftd.feasibility_test_question_id ' + str(Test[0]) + ' & bbftr.base_bmp_feasibility_test_result_id '+ str(Test[3]) + ')')
        is_bb_feasible = is_bb_feasible * Test[2] #bool op to determine bmp feasibility
    print ('  Result: ' + str(bool(is_bb_feasible)))
    return is_bb_feasible
    
def Eval_base_bmp_feasibility_tests(dbFileName, facility_id, base_bmp_id, bmp_name,flgCommitAfterTransaction):
    #evaluate feasibility tests for a single facility in facility_char table & a single base_bmp from the base_bmps table
    print ('Determine Facility-base_bmp feasibility. Unique Identifiers:   Facility ID: ' + str(facility_id) + '  base_bmp_ID: ' + str(base_bmp_id) )
    sql_str = 'SELECT bbftd.feasibility_test_question_id, ftq.question_expression_id, bbftd.base_bmp_feasibility_test_definition_id \
                FROM base_bmp_feasibility_test_definitions AS bbftd \
                INNER JOIN feasibility_test_questions AS ftq \
                ON bbftd.feasibility_test_question_id = ftq.feasibility_test_question_id \
                WHERE base_bmp_id = (?)'
    ParamTuple = (base_bmp_id,)
    myRows = QrySQL(dbFileName, sql_str, ParamTuple)

    for row in myRows:
        #build QryOnUnqFieldValsDict:
        print ('\nattempting eval of feasibility_test: ' + str(row[0]) + '(base_bmp_feas._test ID: ' + str(row[2]) + ')')
        QryOnUnqFieldValsDict = {'facility_chars.facility_id': facility_id,
                                 'base_bmps.bmp_name': bmp_name} #bmp_name is needed b/c the test's expression may be unique to a particular bmp 
#         print (QryOnUnqFieldValsDict)
        sql_str = 'SELECT expression_name, expression_str, expression_data_type, vars \
                    FROM expressions WHERE expression_id = (?)'
        ParamTuple = (row[1],)
        myExpr = QrySQL (dbFileName, sql_str, ParamTuple)
        if len(myExpr)>0:
            myExpr = myExpr[0] #extract tuple from list
            is_feasible = bool(EvalExpr(dbFileName,myExpr, QryOnUnqFieldValsDict))
            print ('Writing to DB Feasibility Test Result: ' + str(is_feasible) + '(' + str(int(is_feasible)) + ')')
            #insert/update feasibility test result:
            RecordFieldLS = ['facility_id','base_bmp_feasibility_test_definition_id', 'is_feasible']
            RecordValsTPL = (facility_id, row[2], int(is_feasible))
            recID = setRecord(dbFileName, 'base_bmp_feasibility_test_results', RecordFieldLS, RecordValsTPL, 'base_bmp_feasibility_test_result_id', \
                              'facility_id = (?) AND base_bmp_feasibility_test_definition_id = (?)',
                             (facility_id, row[2]))
            print ('  Wrote to base_bmp_feasibility_test_results as recordID: ' + str(recID))
        else:
            print ('!!!! FAULT! expression_id: ' + row[1] + ' not found in expressions table. this should not happen.')
            return False

def Eval_ALL_base_bmp_feasibility_tests(dbFileName, facility_id,flgCommitAfterTransaction):
#     wrapper around Eval_base_bmp_feasibility_tests. evals facilities in facility_char table
    print ('\nEvaluating base bmp feasibiilty tests for facility: ' + str(facility_id))
    sql_str = 'SELECT base_bmp_id, bmp_name FROM base_bmps ORDER BY base_bmp_id'
    ParamTuple = ()
    myBMPs = QrySQL(dbFileName, sql_str, ParamTuple)
    for aBMP in myBMPs:
        print ('\nCalling eval_base_bmp_feasibility_tests FOR base_bmp: ' + str(aBMP[0]) + ' ' + aBMP[1])
        Eval_base_bmp_feasibility_tests(dbFileName, facility_id, aBMP[0], aBMP[1],flgCommitAfterTransaction)

        
dbConnection = getDBConnection('_jonhonda_dat\\special_prj\\StrBMPModelDB003')
flgCommitAfterTransaction=False
for row in QrySQL(dbConnection, 'SELECT facility_id FROM facility_chars', ()):
    Eval_ALL_base_bmp_feasibility_tests(dbConnection,row[0],flgCommitAfterTransaction)
if flgCommitAfterTransaction==False: dbConnection.commit() #commit transactions at end if false
dbConnection.close()

In [ ]:
#evaluate base bmp cip and o&m costs
def evalFacility_BaseBMPCosts(dbFileName, facility_id):
    #for the given facility, compute base bmp costs only if base bmp is feasible at facility
    #return dictionary of cip and om costs for feasibile base bmps. Format:
        #{key = base_bmp_id: [cip_cost, om_cost]}
        #cost list is defaulted as NaN values. remains NaN until overwritten by evaluation result. remains NaN if no result obtained. useful for detecting undefined conditions
    print ('\nEvaluating base bmp costs at Facility: ' + str(facility_id))
    sql_str = 'SELECT base_bmp_id, bmp_name, cip_expression_id, om_expression_id FROM base_bmps ORDER BY base_bmp_id'
    ParamTuple = ()
    myBMPs = QrySQL(dbFileName, sql_str, ParamTuple)
    CostDict = {}
    for aBMP in myBMPs:
        if is_base_bmp_feasible(dbFileName,facility_id, aBMP[0]):
            print ('base bmp: ' + str(aBMP[0]) + '  at Facility: ' + str(facility_id) + ' is feasible. estimate costs:')
            QryOnUnqFieldValsDict = {'facility_chars.facility_id': facility_id,
                                     'base_bmps.bmp_name': aBMP[1]} #bmp_name is needed b/c the test's expression may be unique to a particular bmp 
    #         print (QryOnUnqFieldValsDict)
            sql_str = 'SELECT expression_name, expression_str, expression_data_type, vars \
                        FROM expressions WHERE expression_id = (?)'
            print ('  Estimate CIP costs:')
            myExpr = QrySQL (dbFileName, sql_str, (aBMP[2],))
            CostLS = [float('nan'),float('nan')] #use NaNs as the defaults in cost list
            if len(myExpr)>0:
                myVal = EvalExpr(dbFileName,myExpr[0], QryOnUnqFieldValsDict)
                CostLS[0] = myVal #write cip cost to cost list
                print ('CIP Cost: ' + str(myVal))
            print ('  Estimate O&M costs:')
            myExpr = QrySQL (dbFileName, sql_str, (aBMP[3],))
            if len(myExpr)>0:
                myVal = EvalExpr(dbFileName,myExpr[0], QryOnUnqFieldValsDict)
                CostLS[1]=myVal #write om cost to cost list
                print ('O&M Cost: ' + str(myVal)) 
            CostDict[aBMP[0]] = CostLS
    return CostDict
    
# print(evalFacility_BaseBMPCosts('_jonhonda_dat\\special_prj\\StrBMPModelDB003',1))

In [ ]:
fault me
#Build bmp options (combos of base_bmps)
def Make_bmp_fingerprint(base_BMP_components):
    #create fingerprint of the passed list of base_bmp_ids
    #corresponds to bmp_options table's bmp_fingerprint field
    #FORMAT: |bmp_option_base_component_id||bmp_option_base_component_id| w/ id's given in ascending order
    fingerprint = '|' + '|'.join(str(id) + '|' for id in base_BMP_components)
    return fingerprint
# print (Make_bmp_fingerprint([1]))
# print (Make_bmp_fingerprint([1,2]))
# print (Make_bmp_fingerprint([1,2,3]))
    
def Make_bmp_base_option_combo(dbFileName, base_BMP_components):
    #build combo using the passed list of base bmps
    #get base bmp removal rate data
    print ('building bmp combo: ' + ','.join(map(str,base_BMP_components)))
    questionmarks = '?' * len(base_BMP_components)
    sql_str = 'SELECT bmp_removal_rates_id, cip_expression_id, om_expression_id FROM base_bmps WHERE base_bmp_id IN ({})'.format(','.join(questionmarks))
    base_bmp_dat = QrySQL(dbFileName, sql_str, base_BMP_components)
    #slice returned list to obtain removal rates, cip exprs, and om expr ids
    bmp_removal_IDs_LS = [row[0] for row in base_bmp_dat]
    cip_expression_IDs_LS = [row[1] for row in base_bmp_dat]
    om_expression_IDs_LS = [row[2] for row in base_bmp_dat]
    
    #get pollutant removal field names
    FieldNames = getFieldNames(dbFileName, 'pollutant_removal_rates')
    r_Fields = [Field for Field in FieldNames if Field[:2] == 'r_'] #add to list if 1st 2 chars is r_, signifying it's a removal field
    #make pollutant removal record:
    prr = cls_Pollutant_Removal_Rates() #instantiate pollutant_Removal_Rate object
    #write pollutant max vals to object
    for r_Field in r_Fields:
        prr.SetPollutantVal(r_Field[2-len(r_Field):], getOpsValue(dbFileName, 'pollutant_removal_rates', r_Field, 'MAX', 'pollutant_removal_rates_id', bmp_removal_IDs_LS)[0][0])
    PRR_id = prr.registerPRRDB(dbFileName, -1234)
    return setRecord(dbFileName, 'bmp_options', ['bmp_fingerprint','bmp_option_removal_rate_id'], (Make_bmp_fingerprint(base_BMP_components),PRR_id[1]), 'bmp_option_id', '', (),False)
    
def Make_ALL_bmp_base_option_combos (dbFileName):
    import itertools     #https://docs.python.org/3/library/itertools.html
    #make all possible combinations of BMPs by permuting larger and larger sets, starting w/ 1 BMP, then 2, etc.
    #CLEAR component tables:
    sql_str = 'SELECT bmp_option_removal_rate_id FROM bmp_options'
    for row in QrySQL(dbFileName, sql_str, ()):
        ExecuteSQL(dbFileName, 'DELETE FROM pollutant_removal_rates WHERE pollutant_removal_rates_id=(?)', (row[0],),False)
    ExecuteSQL(dbFileName, 'DELETE FROM bmp_option_base_components', (), False)
    ExecuteSQL(dbFileName, 'DELETE FROM bmp_options', (), False)
    dbFileName.commit()
    
    #get base bmps:
    sql_str = ('SELECT base_bmp_id, bmp_name FROM base_bmps ORDER BY base_bmp_id')
    myBMPs_Dict = dict(QrySQL(dbFileName, sql_str, ()))#query and transform to dictionary w/ base_bmp_id as key
    myBMPIDs_LS =list(myBMPs_Dict.keys()) #make list of the base_bmp_ids
    BMPCnt = len(myBMPs_Dict) #count number of BMPs
    #now the magic: make BMP combos of increasing length, starting w/ 1 BMP, then 2, ...
#     print (BMPCnt)
    makeCnt = 0
    for CBOLen in range(1,BMPCnt+1): #+1 so it's inclusive of last count
        print (' Making BMP Combos of length: ' + str(CBOLen))
        combos_LS =  list(itertools.combinations(myBMPIDs_LS,CBOLen))
        for cboi in range(0,len(combos_LS)):
#             print (' building combo: ' + str(combos_LS[cboi]))
            Make_bmp_base_option_combo(dbFileName, list(combos_LS[cboi]))
            makeCnt +=1
    print ('made: ' + str(makeCnt))
import time
start_time = time.time()
dbConnection = getDBConnection('_jonhonda_dat\\special_prj\\StrBMPModelDB003')
Make_ALL_bmp_base_option_combos(dbConnection)
dbConnection.commit() #combo making explicitly ommits commiting transactions to increase insert speed
dbConnection.close()
print ('--- %s execution time in seconds ---' % (time.time() - start_time))

In [ ]:
#evaluate feasible bmp options for each facility
# print(evalFacility_BaseBMPCosts('_jonhonda_dat\\special_prj\\StrBMPModelDB003',1)).
def Eval_FacBMPOption(dbFileName, facility_id, bmp_option_combo, fsbl_bmp_cost_dict):
    #evaluate properties of the facilty bmp option given the list of feasible base bmp combos that comprise the option
    #also passed fsbl_base_bmp_dict which contains cost data for each feasible bmp
        #{key = base_bmp_id: [cip_cost, om_cost]}
    #for now, just eval costs. later we'll include WRS reductions
    #insert results to the facility_bmp_options table
    
    print ('Evaluating bmp option (bmp_ids): ' + str(bmp_option_combo))
    #FIND bmp_option table record:
    fingerprint = Make_bmp_fingerprint(bmp_option_combo)
    sql_str = 'SELECT bmp_option_id, bmp_option_removal_rate_id FROM bmp_options WHERE bmp_fingerprint=(?)'
    bmp_option_rec = QrySQL(dbFileName, sql_str, (fingerprint,))
    #calculate revised WRS score:
    
    #calculate costs:
    sumCIP = sum([fsbl_bmp_cost_dict[base_bmp][0] for base_bmp in bmp_option_combo]) #add CIP costs of component base bmps in the fsb_bmp_cost_dict
    sumOM = sum([fsbl_bmp_cost_dict[base_bmp][1] for base_bmp in bmp_option_combo]) #add om costs of component base bmps in the fsb_bmp_cost_dict
    print (' Costs: CIP=' + str(sumCIP) + ' O&M=' + str(sumOM))
    
     #insert/update record in facility_bmp_options table
    RecordFieldLS = ['facility_id', 'bmp_option_id', 'is_feasible', 'calc_cip_cost', 'calc_om_cost']
    RecordValsTPL = (facility_id, bmp_option_rec[0][0], 1, sumCIP, sumOM)
    rec_id = setRecord(dbFileName, 'facility_bmp_options', RecordFieldLS, RecordValsTPL, 'facility_bmp_option_id', 
              'facility_id=(?) AND bmp_option_id = (?)', 
              (facility_id, bmp_option_rec[0][0]),
              True)
    print (' Inserted/updated evaluated data in bmp_options_table as record: ' + str(rec_id))
    
def Eval_FacBMPOptions(dbFileName, facility_id):
    import itertools     #https://docs.python.org/3/library/itertools.html
    print ('\nEvaluating feasibile BMP Options for facility: ' + str(facility_id))
    fsbl_bmp_cost_dict = evalFacility_BaseBMPCosts(dbConnection,facility_id) #get dict of facilit's feasible base bmps
    combos_LS = [] #initialize bmp combos list 
    for CBOLen in range(1,len(fsbl_bmp_cost_dict)+1): #Populate bmp combos list. +1 so it's inclusive of last count
        combos_LS +=  [list(tpl) for tpl in itertools.combinations(list(fsbl_bmp_cost_dict.keys()),CBOLen)]
    for aCombo in combos_LS: #evaluate each bmp combo
        Eval_FacBMPOption(dbFileName, facility_id, aCombo, fsbl_bmp_cost_dict)
    
    
def Eval_All_FacBMPOptions(dbFileName):
    print ('Evaluating feasibile BMP Options for each facility:')
    for row in QrySQL(dbConnection, 'SELECT facility_id FROM facility_chars', ()):
        Eval_FacBMPOptions(dbFileName, row[0])
    
dbConnection = getDBConnection('_jonhonda_dat\\special_prj\\StrBMPModelDB003')
Eval_All_FacBMPOptions(dbConnection)
dbConnection.close()

In [ ]:



#does repeatedly opening and closing db for each SQL call affect performance?
#try time for 1000 inserts to expression table using open close DB method:

#RESOLUTION: the biggest driver was commiting after each transaction. committing only after final insert reduced time by 
# 2 orders of magnitued!

def getDBConnectionTest(dbFileName):
    # make a db connection
    import sqlite3 as sqlite
    #     dbfilename = '_jonhonda_dat\\special_prj\\StrBMPModelDB003'
    dbConnection = sqlite.connect(dbFileName)
    return dbConnection

def OpenCloseExecuteSQL(dbFileName, sql_str, ParamTuple):
    #execute insert/update/delete and other action type sql commands on the given db
    # attempt to return row that was updated
    import sqlite3 as sqlite
#     print (sql_str)
#     print (ParamTuple)
    dbConnection = getDBConnectionTest (dbFileName)
    cursor = dbConnection.cursor()    
    cursor.execute (sql_str, ParamTuple)
    lastRowID = cursor.lastrowid
    dbConnection.commit()
    dbConnection.close()
    return lastRowID

def PassDBObjExecuteSQL(dbObj, sql_str, ParamTuple, flgCommitAfterTransaction):
    #execute insert/update/delete and other action type sql commands on the given db
    # attempt to return row that was updated
#     print (sql_str)
#     print (ParamTuple)
    dbObj.cursor().execute (sql_str, ParamTuple)
    lastRowID = dbObj.cursor().lastrowid
    if flgCommitAfterTransaction: dbObj.commit()
    return lastRowID

import time
# #try open/close after each insert:
# start_time = time.time()
# for i in range(1000):
#     sql_str = 'INSERT INTO expressions (expression_name, expression_str) Values(?,?)'
#     ParamTuple = ('test' + str(i), 'i')
#     OpenCloseExecuteSQL('_jonhonda_dat\\special_prj\\StrBMPModelDB003', sql_str, ParamTuple)
# print ('--- OpenClose Method: %s execution time in seconds ---' % (time.time() - start_time))

# #try open before 1st insert and stay open till last insert; commit after each insert
# dbConnection = getDBConnectionTest ('_jonhonda_dat\\special_prj\\StrBMPModelDB003')
# start_time = time.time()
# for i in range(1000):
#     sql_str = 'INSERT INTO expressions (expression_name, expression_str) Values(?,?)'
#     ParamTuple = ('test' + str(i), 'i')
#     PassDBObjExecuteSQL(dbConnection, sql_str, ParamTuple, True)
# dbConnection.close()
# print ('--- Stay Open w/ commit after each insert Method: %s execution time in seconds ---' % (time.time() - start_time))

#try open before 1st insert and stay open till last insert; commit at end
dbConnection = getDBConnectionTest ('_jonhonda_dat\\special_prj\\StrBMPModelDB003')
start_time = time.time()
for i in range(1000):
    sql_str = 'INSERT INTO expressions (expression_name, expression_str) Values(?,?)'
    ParamTuple = ('test' + str(i), 'i')
    PassDBObjExecuteSQL(dbConnection, sql_str, ParamTuple, False)
dbConnection.commit()
dbConnection.close()
print ('--- Stay Open w/ commit after all insert Method: %s execution time in seconds ---' % (time.time() - start_time))

